# Review Classification
#### In this project I am going to classify reviews in to positive and negetive reviews. I am going to use Embedding, LSTM and a fully connected layer at the end. For sake of learning I am not going to use any pretrained model. 

### Status: In progress